Drive the probabilistic collocation method using DAKOTA

In [1]:
import os, pickle, shutil, sys, time

### Experiment setup

Define the parameters of the chaos expansion and tweak the methodology used by DAKOTA to execute it. These parameters will be saved in a folder corresponding to timestamp of when the analysis was executed.

In [ ]:
exp_name = "SM_LASSO"

# Run jobs or DAKOTA in parallel?
PARALLEL = True  # if false, uses asynchronous script interface instead of linked
USE_MPI  = False # run dakota with mpich?

variables = [ 
    ## The set of variables about which to perform the PCE
    # symbol, name, [prior, *parameters], offset, meta(dictionary)
    ['logN', 'logN', ['uniform', 1.0, 4.0], 1.0, {}],
    ['logmu', 'logmu', ['uniform', -3., -1.], -3., {}],
    
    ['sigma', 'sigma', ['uniform', 1.2, 3.0], 1.2, {}], 
    ['kappa', 'kappa', ['uniform', 0.0, 1.2], 0.0, {}],
    
    ['logV', 'logV', ['uniform', -2., 1.], -2., {}],
    ['T', 'T', ['uniform', 240., 310.], 240., {}],
    ['P', 'P', ['uniform', 50000., 105000.], 50000., {}],
    ['accom', 'accom', ['uniform', 0.1, 1.0], 0.1, {}],
]

# Number the variables
for i, v in enumerate(variables):
    v.insert(2, i + 1)

##############################
## SIMULATION / DAKOTA SETUP

## PARCEL MODEL SETTINGS

# Name to alias the response functions
responses = [ 'Smax', 'Neq', 'Nkn' ]

# Evaluation script-specific imports
imports = [
    "import numpy as np",
]

# Function setup
function_name = "run_model"
function = """
    import numpy as np
    import parcel_model as pm
        
    N = 10.**logN
    mu =10.**logmu
    V = 10.**logV
    
    use_param = False
    
    output_dt = 1.0
    solver_dt = 10.0
    #z_top = 1000.0
    #t_end = np.min([1800., z_top/V])
    t_end = 1800.
    
    aerosol_modes = [
        pm.AerosolSpecies('aer', 
                           pm.Lognorm(mu=mu, sigma=sigma, N=N),
                           kappa=kappa, bins=250),
    ]
    
    if not fn_toggle:
        if use_param:
        ## Parameterization
            Smax, nact, act_frac = pm.arg2000(V, T, P, 
                                              aerosol_modes, 
                                              accom=accom)
            return np.log10(Smax), np.log10(np.sum(nact)), np.log10(np.sum(nact))
        else:
        ## Parcel Model
            try:
                model = pm.ParcelModel(aerosol_modes, V, T, -0.0, P, 
                                       accom=accom,
                                       truncate_aerosols=True)
                par_out, aer_out = model.run(t_end=t_end, 
                                             output_dt=output_dt,
                                             solver_dt=solver_dt,
                                             max_steps=2000,
                                             solver='cvode', 
                                             output='dataframes',
                                             terminate=True, 
                                             terminate_depth=10.)
                Smax = par_out['S'].max()
                T_fin = par_out['T'].iloc[-1]
                
                ## Compute the activated number
                eq, kn = 0., 0.
                for aer in aerosol_modes:
                    rs = aer_out[aer.species].iloc[-1].values
                    af_eq, af_kn, _, _ = pm.binned_activation(Smax, T_fin,
                                                              rs, aer)
                    # Use binned totals for consistency
                    eq += af_eq*np.sum(aer.Nis*1e-6)
                    kn += af_kn*np.sum(aer.Nis*1e-6)
                
            except:
                print ">>>> (%d) model integration FAILED" % run_id, \
                      " -- ", logmu, logN, logV #, "%r" % e 
                Smax = -1.0

            if Smax < 0: 
                #return -9999.0
                Smax, _, _ = pm.arg2000(V, T, P, aerosol_modes, accom=accom)
                
                return np.log10(Smax), 0., 0.
            else:
                return np.log10(Smax), np.log10(eq), np.log10(kn)
    else:
        Smax_arg, Nact_arg, _ = pm.arg2000(V, T, P, aerosol_modes, 
                                           accom=accom)
        Smax_arg = np.log10(Smax_arg)
        Nact_arg = np.log10(np.sum(Nact_arg))
        
        Smax_mbn, Nact_mbn, _ = pm.mbn2014(V, T, P, aerosol_modes,
                                           accom=accom)
        Smax_mbn = np.log10(Smax_mbn)
        Nact_mbn = np.log10(np.sum(Nact_mbn))
        
        Nderiv, _ = pm.lognormal_activation(fn_toggle, mu*1e-6,
                                            sigma, N, kappa, T=T)
        Nderiv = np.log10(Nderiv)
                                  
        return Smax_arg, Nact_arg, Smax_mbn, Nact_mbn, Nderiv
"""
    

## DAKOTA SETTINGS
graphics = False 

# Analysis options for DAKOTA
pce_directive = """
    askey
    
    {name} = {val}
    tensor_grid
    basis_type total_order
    
    collocation_ratio 3
    #least_angle_regression
    least_absolute_shrinkage
    #least_squares
    
    variance_based_decomp
"""

param_name = "expansion_order"
param_vals = [2, 3, 4, 5]

Save the experiment setup.

In [ ]:
pickle.dump(dict(exp_name=exp_name, param_name=param_name, 
                 param_vals=param_vals, variables=variables,
                 responses=responses, function_name=function_name,
                 directive_base=pce_directive),
            open("%s_exp.dict" % exp_name, 'wb'))

### Generate the model python and DAKOTA driver scripts and run DAKOTA

These scripts will initially reside in the main directory, but will be copied to the `save` archive timestamped folder for referal later.

In [ ]:
print "iterating parameter {name} over vals {vals}".\
       format(name=param_name, vals=param_vals)

path = sys.path
cwd = os.getcwd()
if not (cwd in path): sys.path.append(cwd)

results_dict = {}
for i, param_val in enumerate(param_vals):
    print "   ",  param_val

    directive = pce_directive.format(name=param_name, val=param_val)

    pickle.dump(dict(variables=variables, responses=responses,
                     pce_directive=directive, imports=imports,
                     function_name=function_name,
                     function=function, graphics=graphics,
                     parallel=PARALLEL),
                open('config.p', 'wb'))

    %run gen_scripts.py
    !chmod +x model_run.py

    foldername = !(date +%Y%m%d_%H%M%S)

    foldername = foldername[0]
    print foldername
    results_dict["%s_%r" % (param_name, param_val)] = foldername

    print "Now executing simulation with DAKOTA...",
    if i > 0:
        ## Take advantage of existing restart file and cached
        ## fn evals
        print "reading old restart file"
        if USE_MPI:
            !mpiexec -np 10 dakota -read_restart dakota.rst.all -i model_pce.in > model_pce.out
        else:
            !dakota -read_restart dakota.rst.all -i model_pce.in > model_pce.out
        print "concatenating results into restart file"
        !dakota_restart_util cat dakota.rst dakota.rst.all dakota.rst.new
        !mv dakota.rst.new dakota.rst.all
    else:
        if USE_MPI:
            !mpiexec -np 10 dakota -i model_pce.in > model_pce.out
        else:
            !dakota -i model_pce.in > model_pce.out
        !cp dakota.rst dakota.rst.all
        
    print "complete.\n"

    print "Copying files..."
    save_dir = os.path.join("save", foldername)
    os.mkdir(save_dir)

    for fn in ["model_run.py", 
               "model_pce.in", "model_pce.out", "model_pce.dat",
               "dakota.rst", "config.p"]:
        shutil.copy2(fn, save_dir)
        print fn, "->", save_dir

    print "... done."
    
    time.sleep(1) # to ensure we don't over-write folders!
    
pickle.dump(results_dict, open("%s_results.dict" % exp_name, 'wb'))

### Process output

Analyze the DAKOTA output log to save the chaos expansions, sobol indices,
etc.

In [ ]:
## Process the output of all the runs
results_dict = pickle.load(open("%s_results.dict" % exp_name, 'rb'))

for run_name, foldername in results_dict.iteritems():
    print run_name, foldername
    ## Generate output for sobol, pce
    %run process_output.py {"save/%s" % foldername}

### Sampling

Conduct a global analysis using LHS and save the results. The cell has been re-written to automate sampling over all the chaos expansions, so only run it after all of the chaos expansions have been generated. In general, you should follow these steps:

1. Re-set the IPython cluster.
2. Run the cell once with `n = 10`
3. Set `n` to the actual amount and run it.

If you make changes to any of the utility toolkits (`dist_tools.py`, etc), you may need to reset the entire IPython kernel.

In [5]:
n = 10000
ref_exp = "SM_OLS"
project_arg = "--project"
%run pce_sample --params --parcel {project_arg} -n {n} --parallel --recompute {ref_exp}
ref_design = "%s_LHS_design.csv" % ref_exp

## Reference to previos design available
for exp in ['SM_LARS', 'SM_LASSO']:
#for exp in ['SM_LASSO']:
    new_design = "%s_LHS_design.csv" % exp
    !cp {ref_design} {new_design}
    %run pce_sample --reference {new_design} -n {n} --parallel --recompute {exp}

10000/10000 tasks finished after   26 s
done
done.
Saving to disk at SM_LASSO_LHS_design_results.csv
